# Checkpointing

## Introduction

**Disclaimer**
*This example assumes you have already worked through the CodeWalkthrough notebook and have a decent understanding of how each part fits together.*

During high dimensional (computationally heavy) Bayesian analysis one often must run sampling and/or evidence estimation for long periods of time. One issue often encountered is that the computing facilities may go offline during this period, thus discarding any values computed until this disconnection. To avoid this issue Harmonic supports what is called *Checkpointing* which allows the user to periodically *back-up* the progress of the computation. This example should hopefully illustrate how one may do this in practise.




## Basic problem setup steps

1. As always import relevant python modules:

In [ ]:
import numpy as np
import emcee
import matplotlib.pyplot as plt
from matplotlib import cm
from functools import partial
import sys
sys.path.insert(0, '/Users/matt/Downloads/Software/harmonic')
import harmonic as hm
sys.path.append("../examples")
import utils

2. Define a Bayesian Posterior Function (here we'll use a simple gaussian example)

In [ ]:
def ln_analytic_evidence(ndim, cov):
    """
    Compute analytic ln_e evidence.
    Args: 
        - ndim: 
            Dimensionality of the multivariate Gaussian posterior
        - cov
            Covariance matrix dimension nxn.           
    Returns:
        - double: 
            Value of posterior at x.
    """
    ln_norm_lik = -0.5*ndim*np.log(2*np.pi)-0.5*np.log(np.linalg.det(cov))   
    return -ln_norm_lik

def ln_Posterior(x, inv_cov):
    """
    Compute log_e of n dimensional multivariate gaussian 
    Args: 
        - x: 
            Position at which to evaluate prior.         
    Returns:
        - double: 
            Value of posterior at x.
    """
    return -np.dot(x,np.dot(inv_cov,x))/2.0   

def init_cov(ndim): 
    """
    Initialise random diagonal covariance matrix.
    Args: 
        - ndim: 
            Dimension of Gaussian.        
    Returns:
        - cov: 
            Covariance matrix of shape (ndim,ndim).
    """

    cov = np.zeros((ndim,ndim))
    diag_cov = np.ones(ndim)
    np.fill_diagonal(cov, diag_cov)
    
    return cov

where the final function init_cov is used to randomly assign a diagonal covariance proportional to the identiy matrix.

3. Define parameters for emcee and harmonic machine learning model

In [ ]:
# Define parameters for emcee sampling
ndim = 10                   # number of dimensions
nchains = 200               # total number of chains to compute
samples_per_chain = 5000    # number of samples per chain
nburn = 2000                # number of samples to discard as burn in

# Initialize random seed
np.random.seed(10)

# Hypersphere hyperparameters
max_r_prob = np.sqrt(ndim-1)
domains_sphere = [max_r_prob*np.array([1E0,2E1])]
hyper_parameters_sphere = [None]

# Create covariance matrix 
cov = init_cov(ndim)
inv_cov = np.linalg.inv(cov) 

for absolute simplicity lets use the hypersphere model and fix the radius *i.e.*

In [ ]:
model = hm.model.HyperSphere(ndim, domains_sphere)
model.set_R(np.sqrt(ndim)) # A good rule of thumb for diagonal covaraince Gaussians.
model.fitted = True

Now we need to run the sampler to collect samples but we wish to checkpoint periodically to protect against system crashes. One simple way to do this is to execute the following loop

In [ ]:
# Set initial random position and state
pos = np.random.rand(ndim * nchains).reshape((nchains, ndim)) * 0.1   
rstate = np.random.get_state()

# Define how often you want to be benchmarking the evidence class
chain_is = 10

# Instantiate the evidence class
cal_ev = hm.Evidence(nchains, model)

for chain_i in range(chain_is):
    # Run the emcee sampler from previous endpoint
    sampler = emcee.EnsembleSampler(nchains, ndim, ln_Posterior, args=[inv_cov])
    (pos, prob, rstate) = sampler.run_mcmc(pos, samples_per_chain/chain_is, rstate0=rstate)
    
    # Collect and format samples
    samples = np.ascontiguousarray(sampler.chain[:,:,:])
    lnprob = np.ascontiguousarray(sampler.lnprobability[:,:])
    chains = hm.Chains(ndim)
    chains.add_chains_3d(samples, lnprob)
    
    # 1) Deserialize the Evidence Class
    if chain_i > 0:
        cal_ev = hm.Evidence.deserialize(".temp.gaussian_example_{}.dat".format(ndim))
    
    # 2) Add these new chains to Evidence class
    cal_ev.add_chains(chains)
    
    # 3) Serialize the Evidence Class 
    cal_ev.serialize(".temp.gaussian_example_{}.dat".format(ndim))
    
    # Clear memory 
    del chains, samples, lnprob, sampler, prob

### Compute the Bayesian evidence

Finally we simply compute the Learnt Harmonic Mean estimator by:

In [ ]:
ln_evidence, ln_evidence_std = cal_ev.compute_ln_evidence()

# Also lets compute the analytic evidence whilst we're at it.
ln_evidence_ana = ln_analytic_evidence(ndim, cov)

### Error analysis

Lets take a look at what the error on our estimate is!

In [ ]:
print('Analytic value: {} || Learnt Harmonic Mean value: {}'.format(ln_evidence_ana, ln_evidence))

# Compare to the learnt harmonic mean estimate 
abs_error = np.abs(ln_evidence_ana - ln_evidence)
percent_error = 100.0 * abs_error / ln_evidence_ana

# Compare the variance provided by the learnt harmonic mean estimate
sigma_error = np.log(np.exp(ln_evidence) + np.exp(ln_evidence_std)) - ln_evidence

sigma_count = abs_error / sigma_error

print('Number of sigma away from true value: {}'.format(sigma_count))

# N.B: sigma_count represents the number of standard deviations between the estimated 
#     and analytic results.